<a href="https://www.kaggle.com/code/sujatharamakrishnan/optimal-angle-selector?scriptVersionId=241718638" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Instructions for viewing in Kaggle
1. Input the configuration 
2. Hover over the plot to identify the RA and DEC values ideal for your chosen geometry.
3. You can view contents of the source code by expanding the code cells.

In [2]:
!pip install healpy > /dev/null 2>&1

In [10]:
import healpy as hp
r_lightcone = 45.00  ## corresponding to redshift 10
import numpy as np
coverage_in_sq_degree = 50
BoxL = 20
cos_theta =  1-(coverage_in_sq_degree/2)*np.pi/180**2 ## theta is the angle made with the cone
theta = np.arccos(cos_theta)
def H(i,j,k,x, y, r_lightcone_by_BoxL,theta,flag=0):
    """
    Generate a 2D spherical top-hat function.
    
    Parameters:
        x (ndarray): Array of x-coordinates.
        y (ndarray): Array of y-coordinates.
        radius (float): Radius of the spherical top-hat.
    
    Returns:
        ndarray: 2D array representing the spherical top-hat function.
    """
    if flag == 0:
        X, Y = np.meshgrid(x, y)  ## here  X is dec , Y is ra
    elif flag == 1:
        X, Y = x,y
    cosdistance = k*np.sin(X) + np.cos(X)*(np.cos(Y)*i+np.sin(Y)*j)
    cosradius = ((r_lightcone_by_BoxL)*np.sin(2*theta)*np.sin(theta)+np.cos(theta)*np.sqrt(i**2+j**2+k**2-(r_lightcone_by_BoxL)**2*np.sin(2*theta)**2))
    # ~ mask  = cosdistance>= np.cos(radius)
    mask  = cosdistance>= cosradius
    return mask 


def analytic_mask(ra,dec,r_lightcone_by_BoxL,theta,flag=0):
    N = int(np.ceil(r_lightcone_by_BoxL)) ## make larger if required
    i,j,k = 0,0,0
    if flag==0:
        top_hat = np.zeros([n,n])
    elif flag==1:
        top_hat = np.zeros([len(ra)])
    for i in range(-N,N):
    	for j in range(-N,N):
    		for k in range(-N,N):
    			if ((i**2+j**2+k**2)<(r_lightcone_by_BoxL)**2):
    				top_hat += H(i,j,k,(dec), (ra), r_lightcone_by_BoxL,theta,flag=flag)
    return top_hat
nside = 32
npix = hp.nside2npix(nside)
theta0, phi0 = hp.pix2ang(nside, np.arange(npix))
# Convert to RA and Dec
RA = (phi0)  # Phi is the azimuthal angle (longitude)
DEC = np.pi/2 - (theta0)  # Theta is the polar angle (co-latitude)
import numpy as np
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt



def plot_mask(r_lightcone_by_BoxL, theta):
    Z = analytic_mask(RA, DEC, r_lightcone_by_BoxL, theta, flag=1)
    Z = np.clip(Z, 0, 1)
    fig = go.Figure(data=go.Scattergeo(lon=np.degrees(RA) - 180,  # Shift for Mollweide plot (Plotly expects -180 to 180)
                                       lat=np.degrees(DEC),mode='markers',marker=dict(size=7,symbol='diamond',color=Z,cmin=0,cmax=1,
        #colorbar=dict(title='Mask Value'),
        colorbar=dict(tickvals=[0, 1],ticktext=['Unique\nstructures', 'Repeated\nstructures'],x=-0.1,len=0.5), 
        colorscale=[[0, '#FEE08B'], [1, '#999999']],  # grey → green
        reversescale=False,
        showscale=True),
    text=[f"RA: {ra:.2f}°, DEC: {dec:.2f}°, Value: {val:.3f}" for ra, dec, val in zip(np.degrees(RA), np.degrees(DEC), np.degrees(Z))],
    hoverinfo='text'))

    fig.update_geos(
    projection_type="mollweide",
    showcountries=False,
    showcoastlines=False,
    showland=False,
    lataxis_showgrid=True,
    lonaxis_showgrid=True)

    fig.update_layout(
    title=(
        ""
    ),
    width=850,
    height=650,
    margin=dict(l=0, r=0, t=40, b=0))
    fig.show()
    print ('done')

def main():
    import math

    # Ask user for input values
    r_lightcone = float(input("Enter length of the lightcone (e.g., 450 Mpc): "))
    BoxL = float(input("Enter length of your simulation box (e.g., 300 Mpc): "))  # Fixed box size as before
    coverage_in_sq_degree = float(input("Enter sky coverage in square degrees (e.g., 50): "))

    # Compute theta from coverage_in_sq_degree
    cos_theta = 1 - (coverage_in_sq_degree / 2) * np.pi / 180**2
    theta = np.arccos(cos_theta)

    # Compute r_lightcone_by_BoxL
    r_lightcone_by_BoxL = r_lightcone / BoxL

    #print(f"Using r_lightcone_by_BoxL = {r_lightcone_by_BoxL:.3f} and theta = {np.degrees(theta):.3f} degrees")

    # Now plot the mask
    plot_mask(r_lightcone_by_BoxL, theta)


if __name__ == "__main__":
    main()


Enter length of the lightcone (e.g., 450 Mpc):  450
Enter length of your simulation box (e.g., 300 Mpc):  300
Enter sky coverage in square degrees (e.g., 50):  50


done
